# Which events were the most impactful?

#### When did they occur? What type were they? Where did they occur?
#### Quantify the damages to people and property

In [2]:
# Load in the stuffs
import pandas as pd
import pyexasol

pd.set_option("display.max_colwidth", None)
def print_query(query, connection):
    stmt = connection.export_to_pandas(query)
    print(stmt.head())
    return stmt

host = '192.168.56.101/3D518979E1BE8B4E0A23B7A296E6AD35D5EC29DCCE01E5CC6ECBA7C9E38D8E96:8563'  # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)

#### Overall Human damage

In [13]:
# Top 10 most destructive events in number of injuries + deaths both direct and indirect
q = '''
SELECT 
    T.EVENT_ID, 
    T.EVENT_TYPE,
    T.REGION,
    T.BEGIN_DATE_TIME,
    T.END_DATE_TIME,
    SUM(T.INJURIES_DIRECT) AS SUM_INJURIES_DIRECT,
    SUM(T.INJURIES_INDIRECT) AS SUM_INJURIES_INDIRECT,
    SUM(T.DEATHS_DIRECT) AS SUM_DEATHS_DIRECT,
    SUM(T.DEATHS_INDIRECT) AS SUM_DEATHS_INDIRECT,
    SUM(COALESCE((T.INJURIES_DIRECT + T.INJURIES_INDIRECT + T.DEATHS_DIRECT + T.DEATHS_INDIRECT), 0)) AS HUMAN_DAMAGE,
    SUM(COALESCE((T.DAMAGE_PROPERTY + T.DAMAGE_CROPS), 0)) AS NON_HUMAN_DAMAGE
FROM AOL_SCHEMA.WEATHER_EVENTS AS T
GROUP BY 
    T.EVENT_ID,
    T.BEGIN_DATE_TIME,
    T.END_DATE_TIME,
    T.EVENT_TYPE,
    T.REGION
ORDER BY
    HUMAN_DAMAGE DESC;
'''

df = print_query(q, conn)
df

   EVENT_ID EVENT_TYPE     REGION             BEGIN_DATE_TIME  \
0   5506267    Tornado   Missouri  2006-04-02 17:54:00.000000   
1   5500766    Tornado  Tennessee  2006-04-07 13:12:00.000000   
2   5506368    Tornado  Tennessee  2006-04-02 18:32:00.000000   
3   5506369    Tornado  Tennessee  2006-04-02 19:09:00.000000   
4   5506240    Tornado   Arkansas  2006-04-02 16:46:00.000000   

                END_DATE_TIME  SUM_INJURIES_DIRECT  SUM_INJURIES_INDIRECT  \
0  2006-04-02 18:25:00.000000                  130                      0   
1  2006-04-07 13:32:00.000000                  121                      0   
2  2006-04-02 18:55:00.000000                   70                      0   
3  2006-04-02 19:31:00.000000                   42                      0   
4  2006-04-02 17:37:00.000000                   47                      0   

   SUM_DEATHS_DIRECT  SUM_DEATHS_INDIRECT  HUMAN_DAMAGE  NON_HUMAN_DAMAGE  
0                  2                    0           132               

,EVENT_ID,EVENT_TYPE,REGION,BEGIN_DATE_TIME,END_DATE_TIME,SUM_INJURIES_DIRECT,SUM_INJURIES_INDIRECT,SUM_DEATHS_DIRECT,SUM_DEATHS_INDIRECT,HUMAN_DAMAGE,NON_HUMAN_DAMAGE
0,5506267,Tornado,Missouri,2006-04-02 17:54:00.000000,2006-04-02 18:25:00.000000,130,0,2,0,132,0
1,5500766,Tornado,Tennessee,2006-04-07 13:12:00.000000,2006-04-07 13:32:00.000000,121,0,7,0,128,0
2,5506368,Tornado,Tennessee,2006-04-02 18:32:00.000000,2006-04-02 18:55:00.000000,70,0,16,0,86,0
3,5506369,Tornado,Tennessee,2006-04-02 19:09:00.000000,2006-04-02 19:31:00.000000,42,0,6,0,48,0
4,5506240,Tornado,Arkansas,2006-04-02 16:46:00.000000,2006-04-02 17:37:00.000000,47,0,0,0,47,0
...,...,...,...,...,...,...,...,...,...,...,...
25729,5489140,Winter Weather,Massachusetts,2006-01-03 06:45:00.000000,2006-01-03 11:00:00.000000,0,0,0,0,0,0
25730,5501121,Hail,Kentucky,2006-04-07 18:53:00.000000,2006-04-07 18:53:00.000000,0,0,0,0,0,0
25731,5512844,Hail,South dakota,2006-05-02 15:45:00.000000,2006-05-02 15:45:00.000000,0,0,0,0,0,0
25732,5505210,Hail,Pennsylvania,2006-05-30 18:55:00.000000,2006-05-30 18:55:00.000000,0,0,0,0,0,0


#### Overall Non-Human damage

In [12]:
# top 10 most damaging events to property and crops in USD
q = '''
SELECT 
    T.EVENT_ID, 
    T.EVENT_TYPE,
    T.REGION,
    T.BEGIN_DATE_TIME,
    T.END_DATE_TIME,
    SUM(T.INJURIES_DIRECT) AS SUM_INJURIES_DIRECT,
    SUM(T.INJURIES_INDIRECT) AS SUM_INJURIES_INDIRECT,
    SUM(T.DEATHS_DIRECT) AS SUM_DEATHS_DIRECT,
    SUM(T.DEATHS_INDIRECT) AS SUM_DEATHS_INDIRECT,
    SUM(COALESCE((T.INJURIES_DIRECT + T.INJURIES_INDIRECT + T.DEATHS_DIRECT + T.DEATHS_INDIRECT), 0)) AS HUMAN_DAMAGE,
    SUM(COALESCE((T.DAMAGE_PROPERTY + T.DAMAGE_CROPS), 0)) AS NON_HUMAN_DAMAGE
FROM AOL_SCHEMA.WEATHER_EVENTS AS T
GROUP BY 
    T.EVENT_ID,
    T.BEGIN_DATE_TIME,
    T.END_DATE_TIME,
    T.EVENT_TYPE,
    T.REGION
ORDER BY
    NON_HUMAN_DAMAGE DESC 
LIMIT 10;
'''
df = print_query(q, conn)
df

   EVENT_ID EVENT_TYPE REGION             BEGIN_DATE_TIME  \
0   5493085    Drought  Texas  2006-01-01 00:00:00.000000   
1   5490744    Drought  Texas  2006-02-01 00:00:00.000000   
2   5498400    Drought  Texas  2006-03-01 00:00:00.000000   
3   5513319    Drought  Texas  2006-05-01 00:00:00.000000   
4   5503659   Wildfire  Texas  2006-04-11 12:00:00.000000   

                END_DATE_TIME  SUM_INJURIES_DIRECT  SUM_INJURIES_INDIRECT  \
0  2006-01-31 23:59:00.000000                    0                      0   
1  2006-02-28 23:59:00.000000                    0                      0   
2  2006-03-31 23:59:00.000000                    0                      0   
3  2006-05-08 23:59:00.000000                    0                      0   
4  2006-04-13 17:00:00.000000                    2                      0   

   SUM_DEATHS_DIRECT  SUM_DEATHS_INDIRECT  HUMAN_DAMAGE  NON_HUMAN_DAMAGE  
0                  0                    0             0        1000001000  
1                 

,EVENT_ID,EVENT_TYPE,REGION,BEGIN_DATE_TIME,END_DATE_TIME,SUM_INJURIES_DIRECT,SUM_INJURIES_INDIRECT,SUM_DEATHS_DIRECT,SUM_DEATHS_INDIRECT,HUMAN_DAMAGE,NON_HUMAN_DAMAGE
0,5493085,Drought,Texas,2006-01-01 00:00:00.000000,2006-01-31 23:59:00.000000,0,0,0,0,0,1000001000
1,5490744,Drought,Texas,2006-02-01 00:00:00.000000,2006-02-28 23:59:00.000000,0,0,0,0,0,300001000
2,5498400,Drought,Texas,2006-03-01 00:00:00.000000,2006-03-31 23:59:00.000000,0,0,0,0,0,200001000
3,5513319,Drought,Texas,2006-05-01 00:00:00.000000,2006-05-08 23:59:00.000000,0,0,0,0,0,100001000
4,5503659,Wildfire,Texas,2006-04-11 12:00:00.000000,2006-04-13 17:00:00.000000,2,0,0,0,2,90001000
5,5486532,Flood,California,2006-01-01 00:00:00.000000,2006-01-01 07:00:00.000000,0,0,0,0,0,32501000
6,5496255,Wildfire,Texas,2006-03-12 11:30:00.000000,2006-03-18 18:00:00.000000,0,0,0,0,0,27201000
7,5497354,Debris Flow,California,2006-04-01 18:00:00.000000,2006-04-15 12:00:00.000000,0,0,1,0,1,20001000
8,5496254,Wildfire,Texas,2006-03-12 11:00:00.000000,2006-03-18 18:00:00.000000,8,0,12,0,20,18201000
9,5501922,Heavy Rain,California,2006-04-02 15:00:00.000000,2006-04-05 15:00:00.000000,0,0,0,0,0,16601000


#### Most destructive events per region

In [ ]:
# Most destructive events per region with more than 0 human damage factor and more than 0 non-human damage factor
q = '''
WITH RankedEvents AS (
    SELECT 
        T.EVENT_ID, 
        T.EVENT_TYPE,
        T.REGION,
        T.BEGIN_DATE_TIME,
        T.END_DATE_TIME,
        SUM(T.INJURIES_DIRECT) AS SUM_INJURIES_DIRECT,
        SUM(T.INJURIES_INDIRECT) AS SUM_INJURIES_INDIRECT,
        SUM(T.DEATHS_DIRECT) AS SUM_DEATHS_DIRECT,
        SUM(T.DEATHS_INDIRECT) AS SUM_DEATHS_INDIRECT,
        SUM(COALESCE(T.INJURIES_DIRECT, 0) + COALESCE(T.INJURIES_INDIRECT, 0) + COALESCE(T.DEATHS_DIRECT, 0) + COALESCE(T.DEATHS_INDIRECT, 0)) AS HUMAN_DAMAGE,
        SUM(COALESCE(T.DAMAGE_PROPERTY, 0) + COALESCE(T.DAMAGE_CROPS, 0)) AS NON_HUMAN_DAMAGE,
        RANK() OVER(PARTITION BY T.REGION ORDER BY SUM(COALESCE(T.DAMAGE_PROPERTY, 0) + COALESCE(T.DAMAGE_CROPS, 0)) DESC) AS NON_HUMAN_DAMAGE_RANK,
        RANK() OVER(PARTITION BY T.REGION ORDER BY SUM(COALESCE(T.INJURIES_DIRECT, 0) + COALESCE(T.INJURIES_INDIRECT, 0) + COALESCE(T.DEATHS_DIRECT, 0) + COALESCE(T.DEATHS_INDIRECT, 0)) DESC) AS HUMAN_DAMAGE_RANK
    FROM 
        AOL_SCHEMA.WEATHER_EVENTS AS T
    GROUP BY 
        T.EVENT_ID,
        T.BEGIN_DATE_TIME,
        T.END_DATE_TIME,
        T.EVENT_TYPE,
        T.REGION
)
SELECT 
    *
FROM 
    RankedEvents
WHERE 
    (NON_HUMAN_DAMAGE_RANK = 1 
    OR HUMAN_DAMAGE_RANK = 1)
    AND HUMAN_DAMAGE > 0
    AND NON_HUMAN_DAMAGE > 0;
    
    
'''

df = print_query(q, conn)
print(df.shape)
df

   EVENT_ID         EVENT_TYPE        REGION             BEGIN_DATE_TIME  \
0   5502369              Flood  North dakota  2006-04-01 00:00:00.000000   
1   5502719              Flood  North dakota  2006-04-01 00:00:00.000000   
2   5505318         Dust Devil  North dakota  2006-05-07 15:40:00.000000   
3   5512277          Lightning  North dakota  2006-05-09 14:00:00.000000   
4   5509573  Thunderstorm Wind          Ohio  2006-05-25 18:25:00.000000   

                END_DATE_TIME  SUM_INJURIES_DIRECT  SUM_INJURIES_INDIRECT  \
0  2006-04-19 23:59:00.000000                    0                      0   
1  2006-04-22 23:59:00.000000                    0                      0   
2  2006-05-07 15:45:00.000000                    1                      0   
3  2006-05-09 14:00:00.000000                    1                      0   
4  2006-05-25 18:29:00.000000                    1                      0   

   SUM_DEATHS_DIRECT  SUM_DEATHS_INDIRECT  HUMAN_DAMAGE  NON_HUMAN_DAMAGE  \
0  

,EVENT_ID,EVENT_TYPE,REGION,BEGIN_DATE_TIME,END_DATE_TIME,SUM_INJURIES_DIRECT,SUM_INJURIES_INDIRECT,SUM_DEATHS_DIRECT,SUM_DEATHS_INDIRECT,HUMAN_DAMAGE,NON_HUMAN_DAMAGE,NON_HUMAN_DAMAGE_RANK,HUMAN_DAMAGE_RANK
0,5502369,Flood,North dakota,2006-04-01 00:00:00.000000,2006-04-19 23:59:00.000000,0,0,1,0,1,1000,1,1
1,5502719,Flood,North dakota,2006-04-01 00:00:00.000000,2006-04-22 23:59:00.000000,0,0,1,0,1,1000,1,1
2,5505318,Dust Devil,North dakota,2006-05-07 15:40:00.000000,2006-05-07 15:45:00.000000,1,0,0,0,1,1000,1,1
3,5512277,Lightning,North dakota,2006-05-09 14:00:00.000000,2006-05-09 14:00:00.000000,1,0,0,0,1,1000,1,1
4,5509573,Thunderstorm Wind,Ohio,2006-05-25 18:25:00.000000,2006-05-25 18:29:00.000000,1,0,0,0,1,1000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,5495081,Thunderstorm Wind,West virginia,2006-04-07 16:30:00.000000,2006-04-07 16:30:00.000000,2,0,0,0,2,1000,1,1
149,5495084,Thunderstorm Wind,West virginia,2006-04-07 17:00:00.000000,2006-04-07 17:00:00.000000,1,0,0,0,1,1000,1,2
150,5510772,Lightning,Wisconsin,2006-05-29 15:07:00.000000,2006-05-29 15:07:00.000000,5,0,0,0,5,1000,2,1
151,5489374,Avalanche,Wyoming,2006-01-05 13:20:00.000000,2006-01-05 13:20:00.000000,1,0,1,0,2,1000,1,1
